In [15]:
import pandas as pd
import time
import numpy as np
from pandas import ExcelFile

In [16]:
file_loc = '/Users/keeyou/Desktop/Navien/4-2020 Sales Commission (to be detailed).xlsx'

In [28]:
#Find Collections and SAP to make DataFrame and then, return three DataFrames
def returnDataFrame(fileLoc):
    read_summary = pd.read_excel(file_loc, sheet_name = 'Total by Rep (USD Funds)')
    read_detail = pd.read_excel(file_loc, sheet_name = 'SAP')
    read_new = pd.read_excel(file_loc, sheet_name = 'New Product')
    summary = pd.DataFrame(read_summary)
    detail = pd.DataFrame(read_detail)
    newProduct = pd.DataFrame(read_new)
    return summary, detail, newProduct

In [29]:
startTime = time.time()
summary, detail, newProduct = returnDataFrame(file_loc)
print("--- %s seconds ---" % (time.time() - startTime))

       Billing Doc.         PO NO.       S.O Type Desc           PH1 D.  \
0          91662556    UO202118126  Return w/o RGA/SRA  US_WATER HEATER   
1          91662554       6-485979       Fill-in order  US_WATER HEATER   
2          91662554       6-485979       Fill-in order  US_WATER HEATER   
3          91662554       6-485979       Fill-in order  US_WATER HEATER   
4          91662554       6-485979       Fill-in order        US_BOILER   
5          91662554       6-485979       Fill-in order        US_BOILER   
6          91662554       6-485979       Fill-in order   US_OTHER ITEMS   
7          91662554       6-485979       Fill-in order   US_OTHER ITEMS   
8          91662553       P1113992      Standard Order   US_OTHER ITEMS   
9          91662552       P1332697      Standard Order   US_OTHER ITEMS   
10         91662552       P1332697      Standard Order   US_OTHER ITEMS   
11         91662551       S3548072      Standard Order   US_OTHER ITEMS   
12         91662549      

In [56]:
#Sort billing numbers in increasing order in detail (SAP)
detail = detail.sort_values(by=['Billing Doc.'])
#Reset index 
detail = detail.reset_index()

In [57]:
#If the billing number does not start with 9 or the length of it is not equal to 8 then remove those rows.
def filteringBillingNumber(summary, table):
    for i in summary.index:
        billingNum = str(summary['Invoice / Credit Memo'][i])
        if len(billingNum) != 8 and billingNum[0] != "9":
            table = table.append(summary.iloc[i, :])
            summary = summary.drop([i], axis=0)
    summary = summary.reset_index()
    return summary, table

In [58]:
#Find rows corresponding to its billing numbers from detailed (SAP)
def findPosition(detail, billingNum):
    start = 0
    end = detail.index._stop-1
    middle = (end//2)
    #print(start, middle, end)
    count = 0
    while (billingNum != str(detail['Billing Doc.'][start]) 
           and billingNum != str(detail['Billing Doc.'][middle])
           and billingNum != str(detail['Billing Doc.'][end]) 
           and count < 15):
        if billingNum > str(detail['Billing Doc.'][start]) and billingNum < str(detail['Billing Doc.'][middle]):
            end = middle
            middle = ((end + start)//2)
            #print(1)
            #print(start, middle, end)
        else:
            start = middle
            middle = ((end + start)//2)
            #print(2)
            #print(start, middle, end)
        count += 1
    return start, end

In [60]:
def returnRows(billingNum, Type, detail, percentDiscount, amount, discountAmt, rate, ck, depositDate, salesRep):
    total = 0
    count = 0
    totalDiscount = 0
    #Match Type with PH1 D.
    ph1d = ""
    if Type == "WH":
        ph1d = "US_WATER HEATER"
    elif Type == "Boiler":
        ph1d = "US_BOILER"
    elif Type == "Part":
        ph1d = "US_OTHER ITEMS"
     
    rows = pd.DataFrame([], columns=[])
    start, end = findPosition(detail, str(billingNum))
    for i in range(start, end):
        if str(detail['Billing Doc.'][i]) == str(billingNum) and detail['PH1 D.'][i] == ph1d:
            discountedPrice = float(detail['Net amount'][i] - (float(detail['Net amount'][i]) * float(percentDiscount)))
            if pd.isna(percentDiscount):
                discountedPrice = round(float(detail['Net amount'][i]), 2)
            total += detail['Net amount'][i]
            oldOrNew = checkOldNew(detail['Material1'][i], [detail['Material'][i]], newProduct)
            extraCom = 0.0
            if oldOrNew == "New":
                #Extra Commission: 0.005 = 0.5%
                extraCom = round(discountedPrice * 0.005, 2)
            df = pd.DataFrame({'Deposit Date': [depositDate],
                               'Ck#': [ck],
                               'P.O.# / RGA# / Debit Memo #': [detail['PO NO.'][i]], 
                               'Invoice / Credit Memo': [detail['Billing Doc.'][i]], 
                               'Customer': [detail['Sold-to'][i]],
                               'City': [detail['Ship-to city Name'][i]],
                               'State / Prov.': [detail['Ship-to state'][i]],
                               'Sales Rep': [salesRep],#[detail['Sales Rep(Doc)1'][i]],
                               'New/ Old': [oldOrNew],
                               'Item': [detail['Material'][i]], 
                               'Item Desc': [detail['Material1'][i]],
                               'Receipt Amt': [round(float(detail['Total amount'][i]), 2)],
                               'Sales Discount': [round(float(detail['Net amount'][i]) * float(percentDiscount), 2)],
                               'Net Amt': [round(discountedPrice, 2)],
                               'Commission': [round(discountedPrice * rate, 2)],
                               'Extra Comm': [extraCom],
                               'Rate': [rate],
                               'Type': [Type]},#[detail['PH1 D.'][i]]}, 
                               index = [count])
            totalDiscount += float(detail['Net amount'][i]) * float(percentDiscount)
            rows = rows.append(df)
            count += 1
            
    #Set true and false to check if the total amount matches
    accuracy = False
    if round(float(total), 2) == round(float(amount), 2):
        accuracy = True
    accuracyDis = False
    if round(float(totalDiscount), 2) == round(float(discountAmt), 2):
        accuracyDis = True
    if pd.isna(totalDiscount):
        accuracyDis = True
    #Add totalAmount at the end of row
    totalAmount = pd.DataFrame({'Item Desc': ['Total'], 
                                'Receipt Amt': [round(total, 2)], 
                                'Accuracy of Receipt Amt': [str(accuracy)],
                                'Sales Discount': [round(totalDiscount, 2)],
                                'Accuracy of Sales Discount': [str(accuracyDis)]})

    #Append totalAmount row
    rows = rows.append(totalAmount, ignore_index=True)
    return rows

In [61]:
#Check if the product is new or old
def checkOldNew(itemNum, itemDesc, newProduct):
    oldOrNew = "Old"
    for i in newProduct.index:
        if newProduct['Product Number'][i] == itemNum or newProduct['Product Description'][i] == itemDesc:
            oldOrNew = "New"
    return oldOrNew

In [62]:
#HIghlight the summarized row with yellow color
def highlight(row):
    if pd.isna(row.Customer):
        return ['background-color: white'] * len(row.values)
    if pd.isna(row.Item):
        return ['background-color: yellow'] * len(row.values)
    else:
        return ['background-color: white'] * len(row.values)

In [63]:
#summary = summary[summary.Ck != 'Transfer']
startTime = time.time()
table = pd.DataFrame([], columns=['Deposit Date', 'Ck#', 'P.O.# / RGA# / Debit Memo #', 'Invoice / Credit Memo', 
                                  'Customer', 'City', 'State / Prov.', 'Sales Rep', 'New/ Old', 'Item',
                                  'Item Desc', 'Receipt Amt', 'Accuracy of Receipt Amt', 'Sales Discount', 
                                  'Accuracy of Sales Discount', 'Shipping', 'DC/120+ Discount', 'Adj.', 
                                  'Net Amt', 'Commission', 'Rate', 'Extra Comm', 'Notes', 'Type'])
for i in summary.index:
    print(round(i/summary.index._stop*100, 2), "%")
    #End the program when it reaches the end
    if pd.isna(summary['Ck#'][i]):
        break;
    table = table.append(summary.iloc[i, :])
    percentDiscount = summary['Sales Discount'][i] / summary['Receipt Amt'][i]
    amount = float(summary['Receipt Amt'][i])
    amountDis = float(summary['Sales Discount'][i])
    rate = float(summary['Rate'][i])
    ck = summary['Ck#'][i]
    Type = summary['Type'][i]
    depositDate = summary['Deposit Date'][i]
    salesRep = summary['Sales Rep'][i]
    rows = returnRows(summary['Invoice / Credit Memo'][i], Type, detail, percentDiscount, amount, amountDis, rate, ck, depositDate, salesRep)
    
    for j in rows.index:
        table = table.append(rows.iloc[j, :])
table = table.reset_index(drop=True).style.apply(highlight, axis=1)
print("100%")
print("--- %s seconds ---" % (time.time() - startTime))
print("Finished")

0.0 %
0.07 %
0.14 %
0.21 %
0.28 %
0.35 %


C:\Users\Keeyou\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


0.42 %
0.49 %
0.56 %
0.63 %
0.7 %
0.77 %
0.84 %
0.91 %
0.98 %
1.05 %
1.12 %
1.19 %
1.26 %
1.33 %
1.4 %
1.47 %
1.54 %
1.61 %
1.68 %
1.75 %
1.82 %
1.89 %
1.96 %
2.03 %
2.1 %
2.17 %
2.24 %
2.31 %
2.38 %
2.45 %
2.52 %
2.59 %
2.66 %
2.73 %
2.8 %
2.87 %
2.94 %
3.01 %
3.08 %
3.15 %
3.22 %
3.29 %
3.36 %
3.43 %
3.5 %
3.57 %
3.64 %
3.71 %
3.78 %
3.85 %
3.92 %
3.99 %
4.06 %
4.13 %
4.2 %
4.27 %
4.34 %
4.41 %
4.48 %
4.55 %
4.62 %
4.69 %
4.76 %
4.83 %
4.9 %
4.97 %
5.04 %
5.11 %
5.18 %
5.25 %
5.32 %
5.39 %
5.46 %
5.53 %
5.6 %
5.67 %
5.74 %
5.81 %
5.88 %
5.95 %
6.02 %
6.09 %
6.16 %
6.23 %
6.3 %
6.37 %
6.44 %
6.51 %
6.58 %
6.65 %
6.72 %
6.79 %
6.86 %
6.93 %
7.0 %
7.07 %
7.14 %
7.21 %
7.28 %
7.35 %
7.42 %
7.49 %
7.56 %
7.63 %
7.7 %
7.77 %
7.84 %
7.91 %
7.98 %
8.05 %
8.12 %
8.19 %
8.26 %
8.33 %
8.4 %
8.47 %
8.54 %
8.61 %
8.68 %
8.75 %
8.82 %
8.89 %
8.96 %
9.03 %
9.1 %
9.17 %
9.24 %
9.31 %
9.38 %
9.45 %
9.52 %
9.59 %
9.66 %
9.73 %
9.8 %
9.87 %
9.94 %
10.01 %
10.08 %
10.15 %
10.22 %
10.29 %
10.36 %
10.43 %

74.39 %
74.46 %
74.53 %
74.6 %
74.67 %
74.74 %
74.81 %
74.88 %
74.95 %
75.02 %
75.09 %
75.16 %
75.23 %
75.3 %
75.37 %
75.44 %
75.51 %
75.58 %
75.65 %
75.72 %
75.79 %
75.86 %
75.93 %
76.0 %
76.07 %
76.14 %
76.21 %
76.28 %
76.35 %
76.42 %
76.49 %
76.56 %
76.63 %
76.7 %
76.77 %
76.84 %
76.91 %
76.98 %
77.05 %
77.12 %
77.19 %
77.26 %
77.33 %
77.4 %
77.47 %
77.54 %
77.61 %
77.68 %
77.75 %
77.82 %
77.89 %
77.96 %
78.03 %
78.1 %
78.17 %
78.24 %
78.31 %
78.38 %
78.45 %
78.52 %
78.59 %
78.66 %
78.73 %
78.8 %
78.87 %
78.94 %
79.01 %
79.08 %
79.15 %
79.22 %
79.29 %
79.36 %
79.43 %
79.5 %
79.57 %
79.64 %
79.71 %
79.78 %
79.85 %
79.92 %
79.99 %
80.06 %
80.13 %
80.2 %
80.27 %
80.34 %
80.41 %
80.48 %
80.55 %
80.62 %
80.69 %
80.76 %
80.83 %
80.9 %
80.97 %
81.04 %
81.11 %
81.18 %
81.25 %
81.32 %
81.39 %
81.46 %
81.53 %
81.6 %
81.67 %
81.74 %
81.81 %
81.88 %
81.95 %
82.02 %
82.09 %
82.16 %
82.23 %
82.3 %
82.37 %
82.44 %
82.51 %
82.58 %
82.65 %
82.72 %
82.79 %
82.86 %
82.93 %
83.0 %
83.07 %
83.14 %
83.21

In [65]:
startTime = time.time()
table.to_excel("04-2020 (USD Funds).xlsx")
print("--- %s seconds ---" % (time.time() - startTime))

--- 107.2086009979248 seconds ---
